In [1]:
import os
os.environ['STAGE'] = 'local'
import sys
sys.path.append('..')

time: 16.6 ms


In [2]:
import base64
import boto3
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import thriftpy2 as thriftpy
import traceback

from itertools import product
from matplotlib.ticker import ScalarFormatter
from pprint import pprint as pp
from scipy.sparse import csr_matrix
from scipy.spatial import distance
from scipy.stats import kendalltau
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from thriftpy2.protocol import TCyBinaryProtocolFactory
from thriftpy2.utils import deserialize, serialize
from urllib import parse

from job.preprocessors import *
from job.models import *
from job.helpers import *

time: 1.34 s


# Check unenriched data

In [4]:
BUCKET_NAME = 'lnl-snowplow-washington-city-paper'
# BUCKET_NAME = 'lnl-snowplow-alpha-media-holdings-newsday'

time: 25.6 ms


In [4]:
s3 = boto3.resource('s3')
snowplow_activities = []
bodies = []
for obj in s3.Bucket(BUCKET_NAME).objects.all():
    itemname = obj.key
    
    if '2021-01-25' not in itemname:
        continue

    obj = s3.Object(BUCKET_NAME, itemname)
    body = obj.get()['Body'].read()
    bodies.append(body)
    contents = body.split(b'\x0b\x00d\x00\x00\x00')
    if len(contents) == 1:
        input(body)
    # First entry is always empty
    snowplow_activities.extend(contents[1:])

time: 38.5 s


* Requisite fields: "session_date" (datetime.date), "client_id" (str), "external_id" (str), "event_action" (str), "event_category" (str)

In [19]:
import time
from progressbar import ProgressBar
from traceback import print_exc

snowplow_data = []

bar = ProgressBar(max_value = len(snowplow_activities))
collector = thriftpy.load("collector-payload.thrift")
collector_payload = collector.CollectorPayload()
protocol = TCyBinaryProtocolFactory()
for i, payload in enumerate(snowplow_activities):

    t0 = time.perf_counter()
#     bar.update(i)
    try:
        raw_payload = deserialize(collector_payload, payload, protocol)
    except e:
        print(e)
        input(payload)
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print('1. Bad payload', t1 - t0)
        print(payload)
        print()
    else:
        print('1. Good payload', t1 - t0)
        print(payload)
        print()
              
    t0 = t1
    
    querystring = raw_payload.querystring
    network_userid = raw_payload.networkUserId
    params = parse.parse_qs(querystring)
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print('2', t1 - t0)
    t0 = t1
    try:
        if params:
            activity_time = datetime.fromtimestamp(int(params['dtm'][0][:-3]))
            snowplow_data.append({
                "session_date": activity_time.date(),
                "client_id": json.loads(params['co'][0])["data"][0]["data"]["ampClientId"],
                "external_id": params['page'][0],
                "activity_time": activity_time,
                "event_category": None,
                "event_action": None
            })
    except:
        pp(params)
        raise
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print(t1 - t0, len(payload))
    
    
snowplow_df = pd.DataFrame(snowplow_data)

1. Bad payload 2.319030393000048
b'\x0e107.77.203.233\n\x00\xc8\x00\x00\x01w6\xd9\xae\x1e\x0b\x00\xd2\x00\x00\x00\x05UTF-8\x0b\x00\xdc\x00\x00\x00\x11ssc-1.0.1-kinesis\x0b\x01,\x00\x00\x00\x89Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Mobile/15E148 Safari/604.1\x0b\x016\x00\x00\x00 https://washingtoncitypaper.com/\x0b\x01@\x00\x00\x00\x02/i\x0b\x01J\x00\x00\x05\x9dp=web&tv=amp-1.0.1&e=ue&ue_pr=%7B%22schema%22%3A%22iglu%3Acom.snowplowanalytics.snowplow%2Funstruct_event%2Fjsonschema%2F1-0-0%22%2C%22data%22%3A%7B%22schema%22%3A%22iglu%3Adev.amp.snowplow%2Famp_page_ping%2Fjsonschema%2F1-0-0%22%2C%22data%22%3A%7B%22scrollLeft%22%3A0%2C%22scrollWidth%22%3A375%2C%22viewportWidth%22%3A375%2C%22scrollTop%22%3A10327%2C%22scrollHeight%22%3A38369%2C%22viewportHeight%22%3A628%2C%22totalEngagedTime%22%3A280%7D%7D%7D&url=https%3A%2F%2Fwashingtoncitypaper.com%2Farticle%2F288998%2Fhonky-tonk-man%2F&page=Honky%20Tonk%20Man%20-%20Washing

1. Bad payload 4.87992334899991
b'\r173.175.33.31\n\x00\xc8\x00\x00\x01w6\xd9\xb1w\x0b\x00\xd2\x00\x00\x00\x05UTF-8\x0b\x00\xdc\x00\x00\x00\x11ssc-1.0.1-kinesis\x0b\x01,\x00\x00\x00\x8dMozilla/5.0 (iPhone; CPU iPhone OS 14_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) GSA/137.2.345735309 Mobile/18C66 Safari/604.1\x0b\x016\x00\x00\x003https://washingtoncitypaper-com.cdn.ampproject.org/\x0b\x01@\x00\x00\x00\x02/i\x0b\x01J\x00\x00\x06}p=web&tv=amp-1.0.1&e=ue&ue_pr=%7B%22schema%22%3A%22iglu%3Acom.snowplowanalytics.snowplow%2Funstruct_event%2Fjsonschema%2F1-0-0%22%2C%22data%22%3A%7B%22schema%22%3A%22iglu%3Adev.amp.snowplow%2Famp_page_ping%2Fjsonschema%2F1-0-0%22%2C%22data%22%3A%7B%22scrollLeft%22%3A0%2C%22scrollWidth%22%3A375%2C%22viewportWidth%22%3A375%2C%22scrollTop%22%3A3096%2C%22scrollHeight%22%3A10065%2C%22viewportHeight%22%3A737%2C%22totalEngagedTime%22%3A54%7D%7D%7D&url=https%3A%2F%2Fwashingtoncitypaper-com.cdn.ampproject.org%2Fc%2Fs%2Fwashingtoncitypaper.com%2Farticle%2F

NameError: name 'e' is not defined

time: 16 s


In [ ]:
snowplow_df

# Train on Snowplow data

In [8]:
formatted_df = format_ga(snowplow_df)
model = train_model(X=formatted_df, reg=2.319952, n_components=130, epochs=2)
article_names = formatted_df.columns
user_names = formatted_df.index

vector_distance = distance.cdist(model.item_vectors, model.item_vectors, metric="cosine")
vector_order = vector_distance.argsort()
for i, order in enumerate(vector_order):
    source_article = article_names[i]
    # First entry is the article itself, so skip it
    print(source_article)
    print('---')
    for j in order[1:5]:
        recommended_article = article_names[j]
        print(recommended_article)
    print()


NameError: name 'snowplow_df' is not defined

time: 57.7 ms


Compare the network user ID to the amp client ID
dtm is the datetime

# Check enriched data

In [4]:
BUCKET_NAME = 'lnl-snowplow-washington-city-paper'
# BUCKET_NAME = 'lnl-snowplow-alpha-media-holdings-newsday'

time: 25.6 ms


In [ ]:
import time
from progressbar import ProgressBar
from traceback import print_exc
snowplow_data = []
bar = ProgressBar(max_value = len(snowplow_activities))
collector = thriftpy.load("collector-payload.thrift")
collector_payload = collector.CollectorPayload()
protocol = TCyBinaryProtocolFactory()
for i, payload in enumerate(snowplow_activities):
    t0 = time.perf_counter()
#     bar.update(i)
    try:
        raw_payload = deserialize(collector_payload, payload, protocol)
    except e:
        print(e)
        input(payload)
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print('1. Bad payload', t1 - t0)
        print(payload)
        print()
    else:
        print('1. Good payload', t1 - t0)
        print(payload)
        print()
    t0 = t1
    querystring = raw_payload.querystring
    network_userid = raw_payload.networkUserId
    params = parse.parse_qs(querystring)
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print('2', t1 - t0)
    t0 = t1
    try:
        if params:
            activity_time = datetime.fromtimestamp(int(params['dtm'][0][:-3]))
            snowplow_data.append({
                "session_date": activity_time.date(),
                "client_id": json.loads(params['co'][0])["data"][0]["data"]["ampClientId"],
                "external_id": params['page'][0],
                "activity_time": activity_time,
                "event_category": None,
                "event_action": None
            })
    except:
        pp(params)
        raise
    t1 = time.perf_counter()
    if t1 - t0 > 2:
        print(t1 - t0, len(payload))
snowplow_df = pd.DataFrame(snowplow_data)

enriched/good/2021/01/29/18/ProdWashingtonCityPaperSnowplow-StreamEnrichStreams-fUjakewLdXJ2-1-2021-01-29-18-38-18-9ba9d07a-bb8d-478b-a084-3ceb38aa9793.gz


In [ ]:
s3 = boto3.resource('s3')
snowplow_activities = []
bodies = []
for obj in s3.Bucket(BUCKET_NAME).objects.all():
    itemname = obj.key
    if '2021-01-25' not in itemname:
        continue
    obj = s3.Object(BUCKET_NAME, itemname)
    body = obj.get()['Body'].read()
    bodies.append(body)
    contents = body.split(b'\x0b\x00d\x00\x00\x00')
    if len(contents) == 1:
        input(body)
    # First entry is always empty
    snowplow_activities.extend(contents[1:])

# Compare to GA data

In [1]:
ga_df = pd.read_csv('../tmp/wcp_sessions_20210125-20210125_20210125-20210125.csv')

time: 289 ms


In [3]:
ga_df.event_action.value_counts()

25%                              8763
Load                             7437
50%                              5478
Seen                             5390
newsletter modal impression 1     702
newsletter modal impression 3     586
100%                              453
Dismissal                         319
Permanent Dismissal                41
social share                       15
Link Click                          7
newsletter signup                   1
Form Submission                     1
Name: event_action, dtype: int64

time: 22.3 ms


In [ ]:
ga_client_ids = ga_df.client_id.unique()
snowplow_client_ids = snowplow_df.client_id.unique()